In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import nose.tools
# Write your imports here

# Data Visualization and Exploratory Data Analysis Lab
## Visualizing and exploring data. Data mining process as a whole

### Problem 1. Read the dataset (1 point)
You'll be exploring data about people's income. Your task is to understand whether there are significant differences in the lifestyle of lower- vs. higher-income groups.

Read the dataset located [here](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data). The information file is [here](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names). Save it into the variable `income_data`. Change the column names to what you like. The last column is related to the income class.

Get acquainted with the information file well before starting work.

You don't need to clean the dataset.

In [ ]:
income_data = None
income_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")


In [ ]:
income_data.columns = ["age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", \
                      "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", \
                       "native_country", "income_class"]

In [ ]:
nose.tools.assert_is_not_none(income_data)

### Problem 2. High income (1 point)
How many people have high income (over 50 000 USD per year)? Write a function to return the value. The function should accept the dataframe as a parameter. Work with that parameter.

In [ ]:
def get_num_people_with_high_income(dataframe):
    result = dataframe[(dataframe["income_class"] == ' >50K')]
    return result

In [ ]:
# This cell contains hidden tests

### Problem 3. Capital gain: thresholding (1 point)
Plot a histogram of the capital gain. You can see that there are many people with relatively low gains and a few people - with very high gains.

In [ ]:
plt.hist(income_data["capital_gain"],bins = 15)
plt.xlabel("Capital gain")
plt.ylabel("Count")
plt.show()

Write a function which accepts a dataframe and a capital gain value (in USD) and returns how many people are there with **greater than or equal to** that threshold gain.

In [ ]:
def get_num_people_with_higher_gain(dataframe, threshold_gain):
    result = dataframe[(dataframe["capital_gain"] >= threshold_gain)]["capital_gain"].count()
    return result

In [ ]:
nose.tools.assert_equal(get_num_people_with_higher_gain(income_data, 60000), 159)

Create a pie chart of the number of people by marital status.

In [ ]:
statuses = list(income_data["marital_status"].unique())
marital_status =  income_data.groupby(income_data["marital_status"]).size()
plt.pie(labels = statuses, x = [status for status in marital_status], autopct= "%.2f%%")
plt.show()

### Problem 4. Marital status (2 points)
Which type of marital status is the most prominent (i.e. has the most people)? How many are there? Write a function that **calculates and returns the two answers**. 

In [ ]:
def most_prominent_marital_status(dataframe):
    status = ""
    num_people = 0
    marital_status_sort = income_data.groupby(income_data["marital_status"]).size().sort_values(ascending=False)
    num_people = marital_status_sort[0]
    status = marital_status_sort.index[0].strip()
    return (status, num_people)

In [ ]:
(status, num_people) = most_prominent_marital_status(income_data)
nose.tools.assert_not_equal(status, "")
nose.tools.assert_greater(num_people, 10000)

### Problem 5. Age groups (1 point)
Create a histogram of all people's ages. Use the default settings. Add the label "Age" on the x-axis and "Count" on the y-axis.

In [ ]:
ax = income_data["age"].plot(kind="hist",
                         title = "All people's ages")
ax.set_xlabel("Age")
ax.set_ylabel("Count")
plt.show()

Let's get another view of the data. Split the ages into three:
1. Young people: $\text{age} \le 30$
2. Middle-aged people: $30 < \text{age} \le 60$
3. Old people: $60 < \text{age}$

Return the counts in the following function. Which age group has the most people? How many are there?

In [ ]:
def get_num_people_by_age_category(dataframe):
    young, middle_aged, old = (0, 0, 0)
    young = income_data[(income_data["age"] <= 30)]["age"].count()
    middle_aged = income_data[(income_data["age"] > 30) & (income_data["age"] <= 60)]["age"].count()
    old = income_data[(income_data["age"] > 60)]["age"].count()
    return young, middle_aged, old

In [ ]:
young, middle_aged, old = get_num_people_by_age_category(income_data)
nose.tools.assert_greater(young, 0)
nose.tools.assert_greater(middle_aged, 0)
nose.tools.assert_greater(old, 0)

Now we can create a bar chart. Execute the code below to see it.

In [ ]:
plt.title("Distribution of people by age groups")
plt.bar(range(3), [young, middle_aged, old])
plt.xticks(range(3), ["Young", "Middle-aged", "Old"])
plt.ylabel("Count")
plt.show()

### Problem 6. Native country (2 points)
Have a look at the native country of the people. The highest number of people are, as expected, from the US. What country makes for **the second highest** number of people? How many are they? Write a function to **calculate** and return the answer given a dataframe. DO NOT hardcode the answer, e.g. `return "Germany"`.

In [ ]:
def get_second_highest_num_people(dataframe):
    num_people, country = 0, ""
    native_country_groups = income_data.groupby(income_data["native_country"]).size().sort_values(ascending=False)
    num_people = native_country_groups[1]
    country = native_country_groups.index[1].strip()
    return num_people, country

In [ ]:
num_people, country = get_second_highest_num_people(income_data)
nose.tools.assert_greater(num_people, 0)
nose.tools.assert_not_equal(country, "")

### Problem 7. Busiest occupations (2 points)
Which people are most overworked? Group all data by occupation and calculate the mean hours per week for each group.

Write a function that **calculates and returns** all mean hours per week as a `Series`. Sort the results in descending order (most hours to fewest hours).

In [ ]:
# income_data["occupation"] contains whitespace, so let's remove it. This will allow
# nose.tools.assert_almost_equal(hours["Handlers-cleaners"], 37.95, delta = 0.01) to run successfully 
income_data["occupation"] = income_data["occupation"].str.strip()

In [ ]:
def get_mean_working_hours_by_occupation(dataframe):
    result = income_data.groupby(income_data["occupation"])["hours_per_week"].mean().sort_values(ascending=False)
    return result

In [ ]:
hours = get_mean_working_hours_by_occupation(income_data)
print(hours)
nose.tools.assert_almost_equal(hours["Handlers-cleaners"], 37.95, delta = 0.01)

Finally, let's plot a bar chart. Check the values carefully. If your do not match, feel free to edit the chart generation code below.

In [ ]:
plt.figure(figsize = (10, 6))
plt.title("Weekly hours by occupation")
plt.barh(range(len(hours)), hours)
plt.yticks(list(range(len(hours))), hours.index)
plt.show()